In [1]:
import numpy as np

In [2]:
np.__version__

'1.23.5'

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/features.txt"
feature_name_df = pd.read_csv(url, sep='\s+', header=None, 
                              names=['column_index', 'column_name'])

feature_name = feature_name_df.iloc[:, 1].values.tolist()

X_train_url = "https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/X_train.txt"
X_test_url = "https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/X_test.txt"

X_train = pd.read_csv(X_train_url, sep='\s+', header=None)
X_test = pd.read_csv(X_test_url, sep='\s+', header=None)

In [3]:
X_train.columns = feature_name
X_test.columns = feature_name

In [4]:
y_train_url = "https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/y_train.txt"
y_test_url = "https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/y_test.txt"

y_train = pd.read_csv(y_train_url, sep='\s+', header=None, names=['action'])
y_test = pd.read_csv(y_test_url, sep='\s+', header=None, names=['action'])

- GBM

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings

warnings.filterwarnings('ignore')

In [26]:
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=13)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)

print('ACC : ', accuracy_score(y_test, gb_pred))
print('Fit time : ', time.time() - start_time)

ACC :  0.9389209365456397
Fit time :  883.9422171115875


- 일반적으로 GBM이 성능자체는 랜덤 포레스트보다는 좋다고 알려져 있음
- sklearn의 GBM은 속도가 아주 느린 것으로 알려져 있음

- 이제 gridsearch로 조금 더 찾아보자

In [27]:
# 40분 넘게 걸림. 그래서 중단시킴

# from sklearn.model_selection import GridSearchCV

# params = {
#     'n_estimators' : [100,500],
#     'learning_rate' : [0.05, 0.1]
# }

# start_time = time.time()
# grid = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1, n_jobs=-1)
# grid.fit(X_train, y_train)

# print('Fit time : ', time.time() - start_time)


- XGBoost

In [28]:
# !pip install xgboost
# pip로 하니 계속 오류가 나서 터미널에 conda로 다시 install 해줌

In [5]:
# 아래 코드를 실행하려고 하는데 <Invalid classes inferred from unique values of `y`.>라는 오류가 떴다. 버전 문제라고 한다. 
# 그래서 라벨 인코더를 사용해서 y_train을 다시 잡아줬다

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

/Users/admin/miniconda3/envs/ds_study/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import time
from xgboost import XGBClassifier

start_time = time.time()
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb.fit(X_train.values, y_train) # sklearn의 모듈들은 pandas를 잘 받아들인다. 근데 xgboost는 그렇지 않기 때문에 numpy array형태로 넣어줘야한다

print('Fit time : ', time.time() - start_time)

In [31]:
y_train = le.inverse_transform(y_train)

In [32]:
accuracy_score(y_test, xgb.predict(X_test.values)) # 아 몰라 성능 겁나 이상하게 나옴

0.02341364099083814

- xgboost는 조기종료 조건과 검증데이터를 지정할 수 있다

In [34]:
# 강의랑 똑같이 했는데 왜 오류나는지 모름

# evals = [(X_test.values, y_test)]

# start_time = time.time()
# xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
# xgb.fit(X_train.values, y_train, early_stopping_rounds=10, eval_set=evals) # early_stopping_rounds : 같은 성능으로 n번 이상 나오면 조기종료, eval_set : 검증데이터

# print('Fit time : ', time.time() - start_time)

- lightGBM
- 데이터가 많으면(10000건 이상) 성능이 좋지만 데이터가 적으면 성능이 아주 안좋다
- 그대신 빠르디

In [35]:
# !pip install lightgbm 
# pip로 하니 계속 오류가 나서 터미널에 conda로 다시 install 해줌

In [ ]:
import time
from lightgbm import LGBMClassifier

evals = [(X_test.values, y_test)]

start_time = time.time()
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(X_train.values, y_train, early_stopping_rounds=100, eval_set=evals) 

print('Fit time : ', time.time() - start_time)